In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Homework 6

**Problem 1**) (5 pts) 6.1.8

**Problem 2**) (5 pts) 6.3.11

**Problem 3**) (5 pts) 6.5.7

**Problem 4**) (5 pts) 6.5.15

**Problem 5**) (5 pts) 6.7.4